In [8]:
import json 
import os
import numpy as np
import random
from tqdm import tqdm
from pathlib import Path
import math

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

#from sklearn.manifold import TSNE
#from openTSNE import TSNE
import umap.umap_ as umap

import cv2
import matplotlib.pyplot as plt

from scipy.special import softmax

In [9]:
# load stimuli info with model priority
DIR_STIMULI = '../exp-source/exp_datasource/screen_images/'

templates = np.load("../checkpoints/template/d_resnet50_ten_template_features_from1000.npz", allow_pickle=True)['arr_0'].item()

display_info =  np.load('../exp-source/display_info.npy', allow_pickle=True)
print('# of total stimuli: ', len(display_info))
display_info[0]

# of total stimuli:  2000


{'display_idx': 50,
 'display_filename': 'TP_dog_d50',
 'condition': 'TP',
 'target_label': 'dog',
 'target_index': 0,
 'clean_objlist': ['../data/imagenet_val_segmented_resized/nontarget/n03980874_ILSVRC2012_val_00017658.JPEG',
  '../data/imagenet_val_segmented_resized/nontarget/n04192698_ILSVRC2012_val_00036370.JPEG',
  '../data/imagenet_val_segmented_resized/nontarget/n07860988_ILSVRC2012_val_00033051.JPEG',
  '../data/imagenet_val_segmented_resized/dog/n02107574_ILSVRC2012_val_00005587.JPEG'],
 'foveated_objlist': ['../data/imagenet_val_segmented_resized_foveated/nontarget/TP_dog_d50-b0-n03980874_ILSVRC2012_val_00017658.JPEG',
  '../data/imagenet_val_segmented_resized_foveated/nontarget/TP_dog_d50-b1-n04192698_ILSVRC2012_val_00036370.JPEG',
  '../data/imagenet_val_segmented_resized_foveated/nontarget/TP_dog_d50-b2-n07860988_ILSVRC2012_val_00033051.JPEG',
  '../data/imagenet_val_segmented_resized_foveated/dog/TP_dog_d50-b3-n02107574_ILSVRC2012_val_00005587.JPEG'],
 'target_idx_in_ob

In [20]:
IMAGE_TYPE  = 'foveated' # 'blurred', 'foveated', 'clean'
#our_results =  np.load(f'../exp-source/our_results_{IMAGE_TYPE}.npy', allow_pickle=True)

template_results = []
for target in TARGET_CATEGORIES:
    display_info_target = list(filter(lambda x: x['target_label'] ==target, display_info))
    n_info = len(display_info_target)
    acc_sum = 0
    n_tp = 0
    for i, info in enumerate(display_info_target):
        imagepathlist = info[f'{IMAGE_TYPE}_objlist']
        features = []
        for img_path in imagepathlist:
            try:
                img = Image.open(img_path).convert('RGB')
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
                continue   
            img_tensor = transform(img).unsqueeze(0).to(device)
            with torch.no_grad():
                feature = model(img_tensor)
            feature = feature.view(feature.size(0), -1).cpu().numpy().squeeze()
            features.append(feature)      
        features = np.array(features)  
        target_template = templates[target][0]
        distances = np.linalg.norm(features - target_template, axis=1)
        probs = softmax(-distances)
        if info['condition'] == 'TP':
            acc = int(np.argmax(probs) == info['target_idx_in_objlist'])
            acc_sum += acc
            n_tp += 1
        else:
            acc = None

        d_results = {}
        d_results['display_idx'] = info['display_idx']
        d_results['display_filename'] = info['display_filename']
        d_results['condition'] = info['condition']
        d_results['target_label'] = info['target_label']
        d_results['objlist'] = imagepathlist
        d_results['target_idx_in_objlist'] = info['target_idx_in_objlist']
        d_results['L2norm'] = distances
        d_results['probs'] = probs
        d_results['acc'] = acc

        template_results.append(d_results)
        #print(features.shape)
        #print(target_template.shape)

    print(f"Target: {target}, # of images: {n_tp}, Accuracy: {acc_sum/n_tp:.2f}")

np.save(f'../exp-source/template_results_{IMAGE_TYPE}.npy', template_results)
print(f'Results saved for {IMAGE_TYPE}')

# Results saved for clean
# Target: dog, # of images: 250, Accuracy: 0.64
# Target: bird, # of images: 250, Accuracy: 0.87
# Target: turtle, # of images: 250, Accuracy: 0.96
# Target: insect, # of images: 250, Accuracy: 0.96

# Results saved for blurred
# Target: dog, # of images: 250, Accuracy: 0.38
# Target: bird, # of images: 250, Accuracy: 0.66
# Target: turtle, # of images: 250, Accuracy: 0.76
# Target: insect, # of images: 250, Accuracy: 0.76

# Results saved for foveated
# Target: dog, # of images: 250, Accuracy: 0.58
# Target: bird, # of images: 250, Accuracy: 0.81
# Target: turtle, # of images: 250, Accuracy: 0.95
# Target: insect, # of images: 250, Accuracy: 0.90



Target: dog, # of images: 250, Accuracy: 0.58
Target: bird, # of images: 250, Accuracy: 0.81
Target: turtle, # of images: 250, Accuracy: 0.95
Target: insect, # of images: 250, Accuracy: 0.90
Results saved for foveated
